The following database diagram illustrates the Chinook database tables and their relationships.

<img src="chinook-er-diagram.png" alt="My Image" style="width:500">

In [ ]:
%pip install ipython-sql

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///chinook.db

1. Provide a query only showing the Customers from Brazil.

In [7]:
%%sql 
SELECT
    FirstName ||' '|| LastName as FullName
    , Country
    , CustomerID
FROM customers
WHERE Country = 'Brazil';

 * sqlite:///chinook.db
Done.


FullName,Country,CustomerId
Luís Gonçalves,Brazil,1
Eduardo Martins,Brazil,10
Alexandre Rocha,Brazil,11
Roberto Almeida,Brazil,12
Fernanda Ramos,Brazil,13


2. Provide a query showing only the Employees who are Sales Agents.

In [11]:
%%sql
SELECT
    *
FROM employees
WHERE Title LIKE '%Sales%Agent%';

 * sqlite:///chinook.db
Done.


EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


3. Find the top 5 genres by the total sales amount.

In [9]:
%%sql
SELECT 
    g.Name AS Genre
    , ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS TotalSales
FROM genres g
JOIN tracks t ON g.GenreId = t.GenreId
JOIN invoice_items ii ON t.TrackId = ii.TrackId
GROUP BY g.Name
ORDER BY TotalSales DESC
LIMIT 5;

 * sqlite:///chinook.db
Done.


Genre,TotalSales
Rock,826.65
Latin,382.14
Metal,261.36
Alternative & Punk,241.56
TV Shows,93.53


4. Find the employee who has generated the highest amount in sales.

In [11]:
%%sql
SELECT 
    e.FirstName
    , e.LastName
    , ROUND(SUM(i.Total), 2) AS TotalSales
FROM employees e
JOIN customers c ON e.EmployeeId = c.SupportRepId
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY e.EmployeeId
ORDER BY TotalSales DESC
LIMIT 1;

 * sqlite:///chinook.db
Done.


FirstName,LastName,TotalSales
Jane,Peacock,833.04


5. Retrieve the most popular artist by total number of tracks sold.

In [12]:
%%sql
SELECT 
    ar.Name
    , COUNT(ii.Quantity) AS TotalTracksSold
FROM artists ar
JOIN albums al ON ar.ArtistId = al.ArtistId
JOIN tracks t ON al.AlbumId = t.AlbumId
JOIN invoice_items ii ON t.TrackId = ii.TrackId
GROUP BY ar.ArtistId
ORDER BY TotalTracksSold DESC
LIMIT 1;

 * sqlite:///chinook.db
Done.


Name,TotalTracksSold
Iron Maiden,140


6. List all the albums that contain tracks of more than one genre.

In [13]:
%%sql 
SELECT 
    al.Title
FROM albums al
JOIN tracks t ON al.AlbumId = t.AlbumId
GROUP BY al.AlbumId
HAVING COUNT(DISTINCT t.GenreId) > 1;

 * sqlite:///chinook.db
Done.


Title
Unplugged
Live After Death
Rock In Rio [CD2]
The Number of The Beast
Greatest Hits
"Battlestar Galactica, Season 3"
"Heroes, Season 1"
"Lost, Season 3"
"Lost, Season 2"
"The Office, Season 3"


7. Find the average length (in milliseconds) of tracks in each media type.

In [15]:
%%sql
SELECT 
    mt.Name AS MediaType
    , ROUND(AVG(t.Milliseconds)) AS AvgTrackLength
FROM media_types mt
JOIN tracks t ON mt.MediaTypeId = t.MediaTypeId
GROUP BY mt.MediaTypeId;

 * sqlite:///chinook.db
Done.


MediaType,AvgTrackLength
MPEG audio file,265574.0
Protected AAC audio file,281724.0
Protected MPEG-4 video file,2342940.0
Purchased AAC audio file,260895.0
AAC audio file,276507.0


8. Find the total sales amount for each customer along with the customer's rank based on their total sales amount.

In [27]:
%%sql
SELECT 
    c.FirstName
    , c.LastName
    , ROUND(SUM(i.Total), 2) AS TotalSales
    , RANK() OVER (ORDER BY SUM(i.Total) DESC) AS SalesRank
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
LIMIT 5;

 * sqlite:///chinook.db
Done.


FirstName,LastName,TotalSales,SalesRank
Helena,Holý,49.62,1
Richard,Cunningham,47.62,2
Luis,Rojas,46.62,3
Ladislav,Kovács,45.62,4
Hugh,O'Reilly,45.62,4


9. For each genre, list the top 3 most expensive tracks.

In [29]:
%%sql 
WITH RankedTracks AS (
    SELECT 
        t.Name AS TrackName
        , g.Name AS GenreName
        , t.UnitPrice
        , ROW_NUMBER() OVER (PARTITION BY g.GenreId ORDER BY t.UnitPrice DESC) AS Rank
    FROM tracks t
    JOIN genres g ON t.GenreId = g.GenreId
)
SELECT TrackName, GenreName, UnitPrice
FROM RankedTracks
WHERE Rank <= 3
LIMIT 9;

 * sqlite:///chinook.db
Done.


TrackName,GenreName,UnitPrice
For Those About To Rock (We Salute You),Rock,0.99
Balls to the Wall,Rock,0.99
Fast As a Shark,Rock,0.99
Desafinado,Jazz,0.99
Garota De Ipanema,Jazz,0.99
Samba De Uma Nota Só (One Note Samba),Jazz,0.99
Enter Sandman,Metal,0.99
Master Of Puppets,Metal,0.99
Harvester Of Sorrow,Metal,0.99


10. Find the genres that account for more than 20% of the total track count in the database.

In [22]:
%%sql
WITH GenreTrackCount AS (
    SELECT 
        g.GenreId
        , g.Name
        , COUNT(t.TrackId) AS TrackCount
    FROM genres g
    JOIN tracks t ON g.GenreId = t.GenreId
    GROUP BY g.GenreId
),
TotalTrackCount AS (
    SELECT COUNT(*) AS TotalTracks
    FROM tracks
)
SELECT gtc.Name, gtc.TrackCount,
       ROUND((gtc.TrackCount * 1.0 / ttc.TotalTracks) * 100, 2) AS PercentageOfTotal
FROM GenreTrackCount gtc
CROSS JOIN TotalTrackCount ttc
WHERE (gtc.TrackCount * 1.0 / ttc.TotalTracks) * 100 > 20;

 * sqlite:///chinook.db
Done.


Name,TrackCount,PercentageOfTotal
Rock,1297,37.03


11. Determine the employees who have supported customers with a combined total spending of over $500.

In [24]:
%%sql 
WITH EmployeeCustomerSpending AS (
    SELECT 
        e.EmployeeId
        , e.FirstName
        , e.LastName
        , ROUND(SUM(i.Total)) AS TotalSpending
    FROM employees e
    JOIN customers c ON e.EmployeeId = c.SupportRepId
    JOIN invoices i ON c.CustomerId = i.CustomerId
    GROUP BY e.EmployeeId
)
SELECT FirstName, LastName, TotalSpending
FROM EmployeeCustomerSpending
WHERE TotalSpending > 500;

 * sqlite:///chinook.db
Done.


FirstName,LastName,TotalSpending
Jane,Peacock,833.0
Margaret,Park,775.0
Steve,Johnson,720.0


12. Identify the top 5 tracks by total sales amount, along with their album and artist names.

In [25]:
%%sql 
WITH TrackSales AS (
    SELECT 
        t.TrackId
        , t.Name AS TrackName
        , al.Title AS AlbumTitle
        , ar.Name AS ArtistName
        , SUM(ii.UnitPrice * ii.Quantity) AS TotalSales
    FROM tracks t
    JOIN albums al ON t.AlbumId = al.AlbumId
    JOIN artists ar ON al.ArtistId = ar.ArtistId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    GROUP BY t.TrackId
)
SELECT TrackName, AlbumTitle, ArtistName, TotalSales
FROM TrackSales
ORDER BY TotalSales DESC
LIMIT 5;

 * sqlite:///chinook.db
Done.


TrackName,AlbumTitle,ArtistName,TotalSales
The Woman King,"Battlestar Galactica, Season 3",Battlestar Galactica,3.98
The Fix,"Heroes, Season 1",Heroes,3.98
Walkabout,"Lost, Season 1",Lost,3.98
Hot Girl,"The Office, Season 1",The Office,3.98
Gay Witch Hunt,"The Office, Season 3",The Office,3.98
